In [ ]:
"""
Mini-project - Visual
For course Deep learning at Aalborg university
Auther: Emil Lytje-dorfman, Jacob Vitfell Køpke, Rune Drongesen
-------
This code runs the a CNN machine learning algorithm sweeeping over the hyperparameters given
to find the best model to find cracks in different surfaces.
Then logs the data to Weights and Biases.
-------
"""

In [ ]:
import os
import sys
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import random
import shutil
from PIL import Image
from sklearn.metrics import confusion_matrix
import seaborn as sns

# Import all necessary PyTorch and Torch Vision packages
import torch
import torchvision
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, random_split
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import SubsetRandomSampler
from torch.utils.data import Dataset
from torchvision.utils import make_grid

# Load the dataset to train for calssifying cracked and non-cracked surfaces
from PIL import Image
from collections import defaultdict

# Load package to measure the time it takes to run the training and test
from pytictoc import TicToc
from tqdm import trange
from tqdm import tqdm

import wandb

import torch
import torch.nn as nn

t = TicToc()


In [ ]:
main_data_path = "archive"
lr = 0.0001
num_epochs = 5
batch_size = 16
dropout = 0.1
# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="Mini_project_2",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": lr,
    "epochs": num_epochs,
    "Batch size": batch_size,
    "Dropout": dropout,
    }
)


In [ ]:
# Function that splits data into a training and test set with user defined ratio
def split_data(data, training_ratio):
  train_size = int(training_ratio * len(data))
  test_size = len(data) - train_size
  train_set, test_set = random_split(data, [train_size, test_size])
  return train_set, test_set

In [ ]:
# Load data for training
data_path = main_data_path + '/walls'
sample_path = 'data_sampled'
sample_images_from_classes(data_path, sample_path, 2000)
data_path = main_data_path + '/Pavements'
sample_images_from_classes(data_path, sample_path, 2000)
data_path = main_data_path + '/Decks'
sample_images_from_classes(data_path, sample_path, 2000)

In [ ]:
# Check if GPU is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
# Define transforms for the image data
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Hyper parameters for the model
lr = [0.00001, 0.0001, 0.001]
num_epochs = [15, 25]
batch_size = [32, 64, 128]
dropout = [0.0, 0.2, 0.5]

In [7]:
# Training loop
for dout in dropout:
    for bs in batch_size:
        for learn_rate in lr:
            for epoch in num_epochs:
                print(f"Dropout: {dout}")
                print(f"Batch Size: {bs}")
                print(f"Learning Rate: {learn_rate}")
                print(f"epochs: {epoch}")

                wandb.init(project="Mini_project_2", config={"learning_rate": learn_rate, "epochs": epoch, "Batch size": bs, "Dropout": dout})
                wandb.config.update({"epochs": epoch}, allow_val_change=True)
                
                # Load data for training
                sampled_data = datasets.ImageFolder(sample_path, transform = transform)
                train_dataset, val_dataset = split_data(sampled_data, 0.8)
                train_loader =  torch.utils.data.DataLoader(train_dataset, batch_size=bs, shuffle=True)
                val_loader =  torch.utils.data.DataLoader(val_dataset, batch_size=bs, shuffle=True)
                
                # Load the pre-trained ResNet-18 model
                resnet = models.resnet18(pretrained=True)
                # Freeze all layers except the last two
                for param in resnet.parameters():
                    param.requires_grad = False
                for param in resnet.layer4.parameters():
                    param.requires_grad = True

                # Modify the last layer for binary classification
                num_ftrs = resnet.fc.in_features
                resnet.fc = nn.Sequential(
                    nn.Linear(num_ftrs, 256),
                    nn.ReLU(),
                    nn.Dropout(dout),
                    nn.Linear(256, 84),
                    nn.ReLU(),
                    nn.Dropout(dout),
                    nn.Linear(84, 1)
                )
                resnet.to(device)
                
                for _ in range(epoch):
                    criterion = nn.BCEWithLogitsLoss()
                    optimizer = optim.Adam(resnet.parameters(), lr=learn_rate)
                    running_loss = 0.0
                    cumu_val_loss = 0.0
                    loss = 0.0
                    outputs = 0.0

                    resnet.train()
                    for (inputs, labels) in tqdm(train_loader, desc="Loop", position=0, leave=True):
                        inputs, labels = inputs.to(device), labels.float().to(device)

                        # Zero the parameter gradients
                        optimizer.zero_grad()
                        
                        # Forward pass
                        outputs = resnet(inputs).squeeze()
                        loss = criterion(outputs, labels)
                        
                        # Backward pass and optimize
                        loss.backward()
                        optimizer.step()
                        
                        running_loss += loss.item() * inputs.size(0)
                    
                    epoch_loss = running_loss / len(train_dataset)

                    resnet.eval()
                    with torch.no_grad():
                        for _, (inputs, labels) in tqdm(enumerate(val_loader), desc="Validation Loop", position=1, leave=True):
                            inputs, labels = inputs.to(device), labels.float().to(device)
                            output = resnet(inputs).squeeze()
                            loss = criterion(output, labels.float())
                            cumu_val_loss += loss.item() * inputs.size(0)

                    val_loss = cumu_val_loss / len(val_dataset)

                    wandb.log({"Training Loss":epoch_loss, 
                               "Validation Loss":val_loss})
                wandb.finish()

# Save the trained model
torch.save(resnet.state_dict(), 'resnet_binary_classification.pth')

Loop: 100%|██████████| 300/300 [00:40<00:00,  7.49it/s]












































































Validation Loop: 75it [00:09,  8.33it/s]
Loop: 100%|██████████| 300/300 [00:39<00:00,  7.60it/s]












































































Validation Loop: 75it [00:09,  8.07it/s]
Loop: 100%|██████████| 300/300 [00:39<00:00,  7.65it/s]












































































Validation Loop: 75it [00:09,  8.01it/s]
Loop: 100%|██████████| 300/300 [00:38<00:00,  7.70it/s]












































































Validation Loop: 75it [00:09,  8.13it/s]
Loop: 100%|██████████| 300/300 [00:38<00:00,  7.71it/s]












































































Validation Loop: 75it [00:08,  8.42it/s]
Loop: 100%|██████████| 300/300 [00:38<00:00,  7.83it/s]












































































Val

Training Loss,█▇▆▅▃▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▁▁▁▁▂▂▃▃▃▄▅▅▆▅▆▅▇▅▇▆▆▇▆█▆
Training Loss,0.02156
Validation Loss,0.91819


Batch Size: 64
Learning Rate: 1e-05
epochs: 15


c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loop: 100%|██████████| 150/150 [00:37<00:00,  4.01it/s]







































Validation Loop: 38it [00:09,  4.21it/s]
Loop: 100%|██████████| 150/150 [00:37<00:00,  4.01it/s]







































Validation Loop: 38it [00:08,  4.23it/s]
Loop: 100%|██████████| 150/150 [00:37<00:00,  4.04it/s]























Training Loss,█▆▆▅▅▄▄▄▃▃▂▂▂▁▁
Validation Loss,█▄▃▂▁▁▁▁▁▂▂▃▄▅▇
Training Loss,0.11063
Validation Loss,0.55198


...

c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loop: 100%|██████████| 75/75 [00:37<00:00,  2.00it/s]




















Validation Loop: 19it [00:08,  2.14it/s]
Loop: 100%|██████████| 75/75 [00:37<00:00,  2.03it/s]




















Validation Loop: 19it [00:08,  2.12it/s]
Loop: 100%|██████████| 75/75 [00:37<00:00,  2.00it/s]




















Validation Loop: 19it [00:08,  2.17it/s]
Loop

Training Loss,█▇▅▄▃▂▂▁▁▁▁▁▁▁▁
Validation Loss,▁▁▁▂▅▄▆▇▅██▇▇▇▆
Training Loss,0.04307
Validation Loss,0.89258


epochs: 25


c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\runed\miniconda3\envs\slml\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Loop: 100%|██████████| 75/75 [00:36<00:00,  2.03it/s]




















Validation Loop: 19it [00:08,  2.16it/s]
Loop: 100%|██████████| 75/75 [00:37<00:00,  2.03it/s]




















Validation Loop: 19it [00:08,  2.13it/s]
Loop: 100%|██████████| 75/75 [00:37<00:00,  1.98it/s]




















Validation Loop: 19it [00:08,  2.17it/s]
Loop

Training Loss,█▇▅▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Validation Loss,▁▁▁▁▂▃▃▄▃▅▅▆█▆▇▆▆▇▆▇▆▆▇▅▅
Training Loss,0.03019
Validation Loss,1.06344
